# Ayudantía 08: Networking 💾

Ayudantes:

- Julio Huerta
- Felipe Vidal
- Diego Toledo
- Alejandro Held
- Clemente Campos

## Serialización

### Introducción
La información en los computadores se guarda en **binario**, también conocidos como bits. Pero para ocupar información más útil utilizamos *bytes* que son compuestos por 8 *bits*. Los bytes son la forma en que se mide típicamente la información en los computadores.

Cuando creamos cierta estructura de datos o un objeto en Python y queremos enviarlo a través de una red o guardarlo en un archivo, necesitamos convertirlo a una secuencia de bytes. A este proceso se le llama **serialización**.

### Pickle
Pickle es un módulo de Python que permite serializar y deserializar objetos de Python. Es muy útil para guardar objetos en archivos o enviarlos a través de la red. Tiene dos funciones principales: `pickle.dumps()` y `pickle.loads()`, las cuales permiten serializar y deserializar objetos respectivamente.

In [3]:
import pickle

lista = [1, (2,3), 4, "hola", "mundo", 3.14]
lista_serializada = pickle.dumps(lista)

print(f'Lista serializada: {lista_serializada}')
print(f'Tipo de dato de la lista serializada: {type(lista_serializada)}')

lista_deserializada = pickle.loads(lista_serializada)
print(f'Lista deserializada: {lista_deserializada}\n')

print(f'¿Son iguales las listas? {lista == lista_deserializada}')
print(f'Son el mismo objeto? {lista is lista_deserializada}')

Lista serializada: b"\x80\x04\x95'\x00\x00\x00\x00\x00\x00\x00]\x94(K\x01K\x02K\x03\x86\x94K\x04\x8c\x04hola\x94\x8c\x05mundo\x94G@\t\x1e\xb8Q\xeb\x85\x1fe."
Tipo de dato de la lista serializada: <class 'bytes'>
Lista deserializada: [1, (2, 3), 4, 'hola', 'mundo', 3.14]

¿Son iguales las listas? True
Son el mismo objeto? False


Se puede observar que transforma la lista en una secuencia de bytes (b"..."), donde cada código de byte representa cierto aspecto de la información. Luego de serializar y deserializar la información, obtenemos una lista con la misma información, pero no son el mismo objeto. 

Se puede guardar la información serializada en un archivo directamente con las funciones `pickle.dump()` y `pickle.load()`. Pickle no es seguro, por lo que no se recomienda usarlo con información de fuentes desconocidas, ya que podría ejecutar código malicioso (Es más, **nunca** debieses cargar con pickle información de fuentes desconocidas).


In [4]:
from os import path

matriz = [[1,2,3], [4,5,6], [7,8,9]]
with open(path.join('data', 'matriz.bin'), 'wb') as file:
    pickle.dump(matriz, file)

with open(path.join('data', 'matriz.bin'), 'rb') as file:
    matriz_cargada = pickle.load(file)


print(f'Matriz original: {matriz}')
print(f'Matriz cargada: {matriz_cargada}')

print(f'¿Son iguales las matrices? {matriz == matriz_cargada}')
print(f'Son el mismo objeto? {matriz is matriz_cargada}')


Matriz original: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
Matriz cargada: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
¿Son iguales las matrices? True
Son el mismo objeto? False


Similarmente al caso de dumps y loads, dump y load permiten serializar y deserializar objetos en archivos y aunque el contenido sea el mismo, no son el mismo objeto.


Al usar pickle con objetos, se busca el método `__getstate__` del objeto y si no existe, tomará el objeto `__dict__` de este. Para deserializar, se buscará el método `__setstate__` y si no existe, se setea el `__dict__` del objeto.

In [7]:
class Auto:
    def __init__(self, marca, modelo, desgaste):
        self.marca = marca
        self.modelo = modelo
        self.uso = 0
        self.desgaste = desgaste

    def __getstate__(self) -> object:
        # Aumentamos la cantidad de usos que ha tenido el auto
        nuevo_estado = self.__dict__.copy()
        print(f"Serializando el auto: {nuevo_estado['marca']} {nuevo_estado['modelo']}")
        nuevo_estado.update({"uso": self.uso+1})
        return nuevo_estado
    
    def __setstate__(self, state):
        print(f"Restaurando el estado del auto, aumentando su desgaste")
        state.update({"desgaste": state["desgaste"] * 1.1})
        self.__dict__ = state


auto = Auto("Toyota", "Corolla", 0.5)
print(f'Auto original: {auto.__dict__}')
auto_serializado = pickle.dumps(auto)
auto_deserializado = pickle.loads(auto_serializado)
print(f'Auto deserializado: {auto_deserializado.__dict__}')

Auto original: {'marca': 'Toyota', 'modelo': 'Corolla', 'uso': 0, 'desgaste': 0.5}
Serializando el auto: Toyota Corolla
Restaurando el estado del auto, aumentando su desgaste
Auto deserializado: {'marca': 'Toyota', 'modelo': 'Corolla', 'uso': 1, 'desgaste': 0.55}


Se utiliza `__getstate__` y `__setstate__` en casos donde el objeto tiene atributos que dependen de las condiciones del programa, como pór ejemplo tenemos un usuario con amistades. Cuando guardamos el objeto, no queremos guardar las amistades, ya que estas pueden cambiar en el tiempo y pueden causar errores al cargarse. 


### JSON
Otra forma de serializar objetos es utilizando JSON (JavaScript Object Notation). JSON es un formato de texto que permite representar objetos de forma sencilla y se puede utilizar en múltiples lenguajes de programación, mientras que pickle está limitado a ser usado en Python. Python tiene un módulo llamado `json` que permite serializar y deserializar objetos a y desde JSON.

La desventaja que tiene respecto a pickle es que JSON solo puede serializar objetos que sean de los siguientes tipos: `dict`, `list`, `tuple`, `str`, `int`, `float`, `bool` y `None`. Por lo que si queremos serializar un objeto que no sea de estos tipos, debemos convertirlo a uno de estos tipos antes de serializarlo.

In [13]:
import json

class Pokemon:
    def __init__(self, nombre, tipo, nivel):
        self.nombre = nombre
        self.tipo = tipo
        self.nivel = nivel
        self.experiencia = 0

poke = Pokemon("Pikachu", "Electrico", 5)

# Serializamos el objeto a un string JSON
poke_json = json.dumps(poke.__dict__)
print(f'Objeto serializado a JSON: {poke_json}')
poke_json_deserializado = json.loads(poke_json)
print(f'Objeto deserializado de JSON: {poke_json_deserializado}, con tipo {type(poke_json_deserializado)}')

Objeto serializado a JSON: {"nombre": "Pikachu", "tipo": "Electrico", "nivel": 5, "experiencia": 0}
Objeto deserializado de JSON: {'nombre': 'Pikachu', 'tipo': 'Electrico', 'nivel': 5, 'experiencia': 0}, con tipo <class 'dict'>


Distinto a pickle, luego de deserializar un objeto, este no es del mismo tipo, sino que es un diccionario. Por lo que si queremos que sea de un tipo específico, debemos convertirlo manualmente.

También se puede personalizar la serialización en JSON, como en pickle, con las funciones de `default` y `object_hook`, pero para que funcione con JSON se debe heredar de `JSONEncoder`.

In [14]:
class PokemonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Pokemon):
            return {
                "nombre": obj.nombre,
                "tipo": obj.tipo,
                "nivel": obj.nivel,
                "experiencia": obj.experiencia,
                "experiencia_acumulada": obj.nivel * 100 + obj.experiencia
            }
        return json.JSONEncoder.default(self, obj)

In [20]:
poke1 = Pokemon("Charmander", "Fuego", 23)
poke2 = Pokemon("Squirtle", "Agua", 10)

json_string = json.dumps(poke1.__dict__)
print(json_string)

json_string = json.dumps(poke1, cls=PokemonEncoder)
print(json_string)
json_string = json.dumps(poke2, cls=PokemonEncoder)
print(json_string)

def pokemon_decoder(dict):
    return Pokemon(dict["nombre"], dict["tipo"], dict["nivel"])

datos = json.loads(json_string)
print(f'Información del pokemon deserializado: {datos}, del tipo {type(datos)}')
datos = json.loads(json_string, object_hook=pokemon_decoder)
print(f'Información del pokemon deserializado: {datos}, del tipo {type(datos)}')

{"nombre": "Charmander", "tipo": "Fuego", "nivel": 23, "experiencia": 0}
{"nombre": "Charmander", "tipo": "Fuego", "nivel": 23, "experiencia": 0, "experiencia_acumulada": 2300}
{"nombre": "Squirtle", "tipo": "Agua", "nivel": 10, "experiencia": 0, "experiencia_acumulada": 1000}
Información del pokemon deserializado: {'nombre': 'Squirtle', 'tipo': 'Agua', 'nivel': 10, 'experiencia': 0, 'experiencia_acumulada': 1000}, del tipo <class 'dict'>
Información del pokemon deserializado: <__main__.Pokemon object at 0x00000173363DAED0>, del tipo <class '__main__.Pokemon'>


### Bytes y encoding
Con pickle guardamos objetos como bytes, pero también podemos crear bytes manualmente y realizar diversas operaciones sobre este formato. Debido a que un `byte` tiene 8 bits, tiene hasta 256 combinaciones distintas. Se puede representar entonces como un número entre 0 y 255 que representen distintos caracteres, como por ejemplo, el byte `65` representa la letra `A` en ASCII, siendo ASCII un estándar de codificación de caracteres.

La codificación define qué byte representa a qué caracter, y existen distintas codificaciones, como ASCII, UTF-8, UTF-16, etc. Se puede obtener el byte correspondiente a un caracter con la función `ord()` y obtener el caracter correspondiente a un byte con la función `chr()` para ASCII.

Además, aunque la función ord muestre el valor decimal, típicamente se trabaja con dos dígitos hexadecimales que permiten describir un byte completo. Para obtener el valor hexadecimal de un byte, se puede utilizar la función `hex()` y para crear uno directamente se puede anteponer `0x` al valor hexadecimal.

In [23]:
print(ord('a'))
print(ord('♠'))
print(ord('~'))

print(chr(197))
print(chr(5824))
print(chr(1176))

lista_caracteres = ['U','·','♠','~','Å','᠀','ո']
for caracter in lista_caracteres:
    print(f'Caracter: {caracter}, código: {ord(caracter)}, hexadecimal: {hex(ord(caracter))}')

97
9824
126
Å
ᛀ
Ҙ
Caracter: U, código: 85, hexadecimal: 0x55
Caracter: ·, código: 183, hexadecimal: 0xb7
Caracter: ♠, código: 9824, hexadecimal: 0x2660
Caracter: ~, código: 126, hexadecimal: 0x7e
Caracter: Å, código: 197, hexadecimal: 0xc5
Caracter: ᠀, código: 6144, hexadecimal: 0x1800
Caracter: ո, código: 1400, hexadecimal: 0x578


### Objeto bytes

En python los bytes son un tipo de dato inmutable que representa una secuencia de bytes. Son similare a un string, pero se anteponen con una b y se representan los caracteres con formato hexadecimal y un "\x" antes de cada caracter. Se puede decodificar los caracteres con distintos encoding, como utf-8, ascii, etc, lo que es relevante ya que obtenemos distintos caracteres dependiendo del encoding.

Se pueden decodificar los bytes con el método `decode()` y se pueden codificar los strings con el método `encode()`. Si se intenta decodificar un byte que no es válido para el encoding, se lanzará una excepción.

In [29]:
caracteres = b"\x63\x6c\x69\x63\x68\xe9"
print(caracteres)
print(caracteres.decode('latin1'))
print(caracteres.decode('utf-8'))

b'clich\xe9'
cliché


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: unexpected end of data

In [30]:
string = "esdrújula"
print(string.encode('utf-8'))
print(string.encode('latin1'))

b'esdr\xc3\xbajula'
b'esdr\xfajula'


### Objeto bytearray

El objeto bytearray es similar a los bytes, pero es mutable. Se puede modificar los bytes de un bytearray, pero no se puede modificar los bytes de un byte. Se puede convertir un bytearray a bytes con la función `bytes()` y se puede convertir un bytes a bytearray con la función `bytearray()`.

El objeto bytearray tiene además diversas funcionalidades útiles, como acceder a ciertas partes con slicing, extenderlo con el método `extend()`, insertar bytes con el método `insert()`, remover bytes con el método `remove()`, etc.

In [35]:
mi_bytearray = bytearray(b"Programar con Python")
print(mi_bytearray)
print(mi_bytearray[10:14])
mi_bytearray.extend(b" es muy divertido")
print(mi_bytearray)
print(mi_bytearray[0])

bytearray(b'Programar con Python')
bytearray(b'con ')
bytearray(b'Programar con Python es muy divertido')
80


Relevante respecto al manejo de bytes son los **chunks**, que son bloques de bytes que se utilizan para enviar información a través de la red. Se pueden enviar chunks de bytes a través de una red y luego reconstruir la información original con estos chunks. Estos chunks permiten separar la información en bloques más pequeños y enviarlos de forma más eficiente.

In [38]:
bytearray_texto = bytearray(b"Hola, muy buenos dias! Estuve investigando sobre los bytes y los bytearray")
TAMANO_CHUNK = 4

for i in range(0, len(bytearray_texto), TAMANO_CHUNK):
    chunk = bytearray_texto[i:i+TAMANO_CHUNK]
    print(chunk)


bytearray(b'Hola')
bytearray(b', mu')
bytearray(b'y bu')
bytearray(b'enos')
bytearray(b' dia')
bytearray(b's! E')
bytearray(b'stuv')
bytearray(b'e in')
bytearray(b'vest')
bytearray(b'igan')
bytearray(b'do s')
bytearray(b'obre')
bytearray(b' los')
bytearray(b' byt')
bytearray(b'es y')
bytearray(b' los')
bytearray(b' byt')
bytearray(b'earr')
bytearray(b'ay')


Al pasar los números int a bytes, se debe especificar el orden de transformación, ya que al representar 8 bits, se puede tener el byte más significativo (el que tiene mayor peso/valor) al principio o al final. Se puede especificar el orden con el parámetro `byteorder` en las funciones `int.to_bytes()` y `int.from_bytes()`. Los dos formatos son `big-endian` y `little-endian`, donde el primero pone el byte más significativo al principio y el segundo al final.

In [47]:
(1320).to_bytes(2, byteorder='big')

b'\x05('

In [48]:
(1320).to_bytes(2, byteorder='little')

b'(\x05'

In [49]:
(1320).to_bytes(6, byteorder='big')

b'\x00\x00\x00\x00\x05('

In [50]:
(1320).to_bytes(6, byteorder='little')

b'(\x05\x00\x00\x00\x00'

Se puede observar como cambia el orden del hexadecimal al pasar de un formato a otro, ya que el byte más significativo pasa a ser el menos significativo y viceversa.

Además, en los siguientes ejemplos se puede observar como el formato de los bytes al obtener un int cambia el valor del int en grandes rangos, mayor mientras más bytes se usen para guardar el número.

In [51]:
int.from_bytes(b'\x05(', byteorder='big')

1320

In [52]:
int.from_bytes(b'\x05(', byteorder='little')

10245

In [53]:
int.from_bytes(b'\x00\x00\x00\x00\x05(', byteorder='big')

1320

In [54]:
int.from_bytes(b'\x00\x00\x00\x00\x05(', byteorder='little')

44001939947520

Finalmente, aquí hay un pequeño resumen para recordar lo que hacen los distintos métodos de encoding:

![resumen](resumen.png)

## Networking: elementos básicos

## IP
Las ip son la forma de identificar de forma única a cada computador, es la forma con la que nosotros vamos a poder buscar una máquina en específico para poder enviar mensajes.

## Puerto
Cuando nos conectamos a cierta máquina mediante la **IP**, necesitamos especificar qué puerto estamos usando. Esto es ya que un computador cuenta con **miles** de puertos y cada uno puede ser utilizado por una aplicación a la vez. De esta forma nosotros podemos saber que nos estamos conectando a la aplicación correcta y podemos comunicarnos de forma efectiva.


## Protocolo de comunicación
Al hacer uso de networking tenemos que hacer uso de un protocolo de comunicación. Esto se puede entender como el estándar que van a cumplir los computadores para enviarse los mensajes. Los protocolos principales son:

- **TCP**: este método prioriza la confiabilidad por sobre la rapidez. Esto quiere decir que nos asegura que **todos los paquetes llegan de forma integra** al receptor. Es útil para cuando la información debe llegar 100% correcta. Por ejemplo, uno no quiere que una tarea enviada en canvas tenga un descuento porque un par de bytes del archivo se envíen de forma incorrecta.
  
- **UDP**: este protocolo prioriza la rapidez por sobre la confiabilidad. Es un protocolo más rápido que TCP pero no asegura que toda la información llegue perfectamente. Es útil cuando la pérdida de un poco de información no sea grave, por ejemplo si estás viendo un video en youtube no es conveniente que el video cargue más lento solo para que unos pocos pixeles se vean del color indicado.

## Cliente
Mucha teoría por hoy 😴. Ahora llevemos esto a la práctica. Para esto el primer paso es crear un socket. Un socket es un objeto de python el cual se puede entender como la vía de entrada y salida de información. 
- ¿Quiero enviar un mensaje? ➜ Socket 
- ¿Quiero recibir un mensaje? ➜ Socket
  
Un socket se genera de la siguiente manera: 

In [1]:
# importamos la librería socket
import socket 

# generamos el objeto
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# el primer elemento de la tupla nos dice si ocuparemos las IP en formato IPv4 o IPv6
# AF_INET -> IPv4
# AF_INET6 -> IPv6

# El segundo elemento de la tupla nos dice si ocuparemos los protocolos TCP o UDP
# SOCK_STREAM ->  TCP
# SOCK_DGRAM -> UDP


Ahora debemos decir a quién nos queremos conectar:

In [2]:
# ponemos una tupla con (ip, puerto) 

host = socket.gethostname()  # en este caso especificamos que la IP será la nuestra
port = 8726  # elegimos el puerto con el que queramos conectarnos 

sock.connect((host, port))

Finalmente para mandar mensajes ocupamos send o sendall

In [3]:

# con send mandamos el mensaje y nos dice cuántos bytes fallaron en enviarse
mensaje = "Bueeenas, aquí mandando mi request para saber lo byts que se envian"
mensaje_bytes = mensaje.encode('utf-8')  # pasamos todo a bytes
enviados_efectivamente = sock.send(mensaje_bytes)
print(f"Logramos enviar efectivamente {enviados_efectivamente} bytes")

Logramos enviar efectivamente 68 bytes


In [4]:
# podemos tener varios sockets a la vez en un código
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostname()
sock.connect((host, port))

# en cambio sendall nos asegura al 100% que los mensajes se van a enviar completos.
mensaje = "Hola servidor, estamos probando como funcionas :)"
mensaje_bytes = mensaje.encode('utf-8')  # pasamos todo a bytes
sock.sendall(mensaje_bytes)

None


Finalmente para saber la respuesta del servidor ocuapos recv especificando cuandos bytes se deben enviar.

In [5]:

data_bytes = sock.recv(4096)  # recibimos hasta 4096 bytes de respuesta (si sobran no importa)
data_str = data_bytes.decode("utf-8")
print(data_str)  # veamos que nos responden

# cerramos la conexión, sino se ocupan recursos en su computador :(
# SIEMPRE CIERREN SUS CONEXIONES
sock.close()


Hola! soy el servidor. Gracias por conectarte
Acabo de recibir el mensaje: Hola servidor, estamos probando como funcionas :)


## Servidor

El código anterior necesita que exista un computador esperando conexiones y que le de una respuesta. Se puede entender que el cliente realiza **requests** y **consume los servicios** del servidor. Ahora vamos a aprender a crear un servidor que espere la llegada de clientes y que responda a las solicitudes que se le realicen.

Como ya se dijo, los sockets son la forma de enviar y recibir mensajes. Así que lo primero va a ser obtener nuestro socket.

In [6]:
import socket
# creamos nuestro socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# conseguimos nuestro propio hostname que va a hacer el trabajo de IP
host = socket.gethostname()

# especificamos el puerto en el que los clientes tendrán que conectarse
port = 5732

# conectamos nuestro socket a la IP y puerto.
sock.bind((host, port))

In [7]:
# el código se queda pegado hasta que llegue un cliente
sock.listen()

# ahora aceptamos a quién llegue

# socket_cliente es el socket para comunicarnos con este cliente en específico
socket_cliente, address = sock.accept()

# aquí va la lógica del servidor

Para obtener una experiencia más realista de cómo es trabajar con sockets, correr los archivos `resumen_cliente.py` y `resumen_servidor.py`.